In [ ]:
# %%capture
# %pip install mistral-common

In [2]:
import os
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent)

In [1]:
from transformers import VoxtralForConditionalGeneration, AutoProcessor
import torch

device = "cuda"
repo_id = "mistralai/Voxtral-Mini-3B-2507"

processor = AutoProcessor.from_pretrained(repo_id)
model = VoxtralForConditionalGeneration.from_pretrained(
    repo_id,
    dtype=torch.bfloat16,
    device_map=device
)

preprocessor_config.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tekken.json:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.38G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

In [4]:
from EmoBox.EmoBox import EmoDataset, EmoEval

dataset = "esd"
fold = 1  # different datasets have different number of folds, which can be find in data/
user_data_dir = "./" # path to EmoBox - FIXED: Changed from "Emobox" to "EmoBox"
meta_data_dir = "EmoBox/data/" # path to data folder - FIXED: Changed from "Emobox" to "EmoBox"
label2idx = {'hap':0, 'sad':1, 'ang':2, 'neu':3} # you may need to define a label to index mapping for your own training, see `data/iemocap/label_map.json`

train = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="train")
test = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="test")

since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 28000 samples, only 28000 exists in data dir EmoBox/data/
load in 7000 samples, only 7000 exists in data dir EmoBox/data/
Num. training samples 28000
Num. valid samples 0
Num. test samples 7000
Using label_map {'Neutral': 'Neutral', 'Angry': 'Angry', 'Happy': 'Happy', 'Sad': 'Sad', 'Surprise': 'Surprise'}
since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 28000 samples, only 28000 exists in data dir EmoBox/data/
load in 7000 samples, only 7000 exists in data dir EmoBox/data/
Num. training samples 28000
Num. valid samples 0
Num. test samples 7000
Using label_map {'Neutral': 'Neutral', 'Angry': 'Angry', 'Happy': 'Happy', 'Sad': 'Sad', 'Surprise': 'Surprise'}


In [5]:
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "audio",
                "path": str(test.data_list[0]['wav']),
            },
            {
                "type": "text",
                "text": "Choose one of the emotions that this audio is portraying between ['Neutral', 'Happy', 'Angry', 'Sad']:"},
        ],
    }
]

inputs = processor.apply_chat_template(conversation)
inputs = inputs.to(device, dtype=torch.bfloat16)

outputs = model.generate(**inputs, max_new_tokens=30)
processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

['Neutral']